# The Hydrogen atom wavefunctions

Paul Nakroshis<br>
Dept. of Physics, University of Southern Maine<br>
pauln at maine dot edu<br>
03 Apr 2019

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from pylab import rcParams
rcParams['figure.figsize'] = 12,10

import seaborn as sns   # makes pretty plots
%matplotlib inline
#sns.set_style("darkgrid", {"grid.linewidth": .5, "axes.facecolor": ".9"})
#sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
#sns.set_context("paper")
#sns.set_context("talk")

In this notebook, I look at the Hydrogen atom wavefunctions which are solutions of Schr&ouml;dinger's equation in spherical coordinates

$$ \left(-\frac{\hbar^2}{2m}\nabla^2 -\frac{e^2}{4\pi\epsilon_0}\frac{1}{r}\right)\psi(r,\theta,\phi) = E\,\psi(r,\theta,\phi),$$

the standard solution method (found in any quantum text) is to use separation of variables to solve for the angular $(\theta, \phi)$ and radial $r$ solutions.
The solutions are given by 
 $$\psi(r,\theta,\phi) = \left\{\sqrt{ \left( \frac{2}{n a_0}\right) \frac{(n-l-1)!}{2n(n+l)!} } e^{-r/na_0} \left( \frac{2r}{na_0} \right)^l \left[ L^{2l+1}_{n-l-1} \left( \frac{2r}{na_0} \right) \right]\right\} Y^m_l(\theta,\phi) $$
where $L^{2l+1}_{n-l-1} \left( \frac{2r}{na_0} \right) $ are the associated Laguerre polynomials, and $Y^m_l(\theta,\phi)$ are the spherical harmonics. The radial piece of the solution is enclosed in braces, the angular piece is simply the spherical harmonics. The quantum numbers determining the state of the electron are $(n,l,m)$, and $a_0$ is the Bohr radius:

$$ a_0 = \frac{4\pi\epsilon_0 \hbar^2}{m_e e^2} = \frac{\hbar}{m_e c \alpha} = 5.2917721067(12)\times 10^{−11}\; \mathrm{m} $$

The point of this notebook is to allow students to be able to visualize the pieces of this solution. The Hydrogen atom wavefunctions involve spherical harmonics and Associate Laguerre polynomials, the latter being foreign to most student's experience as a physics major. 

SciPy includes all the functionality to allow us to easily write code to plot the wavefunctions. The [Associated Laguerre polynomials](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.assoc_laguerre.html) and the [spherical harmonics](https://docs.scipy.org/doc/scipy-0.19.1/reference/generated/scipy.special.sph_harm.html) can be found in the scipy.special library. We import the scipy.special library here:

In [2]:
import scipy.special as sp
from scipy.special import factorial
from scipy.special import sph_harm

## Plot of the radial solution
First we note that the argument, $\left( \frac{2r}{na_0} \right) $, of the Associate Laguerre polynomial that appears in $\psi(r,\theta,\phi)$ is the product of a dimensionless number $\rho\equiv\frac{r}{a_0}$, and the dimensionless number $\frac{2}{n}$ so that when we plot the polynomials we will use the x-axis as $\rho = \frac{r}{a_0}$. The function radial defines the full radial portion:

In [9]:
def radial(ρ,n,l):
    """
    This function computes the radial piece of the hydrogen atom wavefunction
    in terms of the dimensionless parameter rho = r/a0, where a0 is the Bohr radius.
    INPUT:
    rho      :r/a0 
    n        :principal quantum number
    l        :angular momentum quantum number

    RETURNS:
    value of the radial piece of the wavefunction
    """
    a0 = 5.2917721067e-11
    return (2*ρ/n)**l * np.sqrt( factorial(n-l-1)/(n**2 * a0 * factorial(n + l))) * np.exp(-ρ/n) * sp.assoc_laguerre(2*ρ/n, n-l-1, 2*l+1)

def angular(θ, ϕ, l, m):
    """
    This function computes the angular portion (spherical harmonic) for hydrogen for a given l, m. 
    INPUT:
    \theta   :polar angle
    \phi.    :azimuthal angle
    l        :angular momentum quantum number
    m.       :magnetic quantum number

    RETURNS:
    value of the spherical harmonic
    """
    return sph_harm(m, l, θ, ϕ)

def ψ(ρ,θ,ϕ,n,l,m):
    return radial(ρ,n,l)*angular(θ, ϕ, l, m)

def prob_density(ρ,θ,ϕ,n,l,m):
    return np.absolute(ψ(ρ,θ,ϕ,n,l,m))

Now make a plot that uses the nifty widget feature of Jupyter notebooks to make it easy for a user to explore these polynomials:

In [4]:
#import ipywidgets as widgets
from ipywidgets import interactive, IntSlider, interact

def plot_radial(n, l):
    plt.figure()
    ρ = np.linspace(0,20,100)
    plt.plot(ρ, radial(ρ, n, l))
    #plt.ylim(-10,10)
    plt.show()

#n_widget = IntSlider(min=1, max=10, step=1, value=1)
#l_widget = IntSlider(min=0, max=9, step=1)
#def update_l_range(*args):
#    l_widget.max = n_widget.value -1
#n_widget.observe(update_l_range, 'value')
#n_value = BoundedIntText(
#    value=1,
#    min=1,
#    max=10,
#    step=1,
#    description='n:',
#    disabled=False
#)
#l_value = BoundedIntText(
#    value=1,
#    min=0,
#    max=9,
#    step=1,
#    description='l:',
#    disabled=False
#)

#def update_l_value(*args):
#    l_value.max = n_value.value -1
#n_value.observe(update_l_value, 'value')
nmax=10
interactive_plot = interactive(plot_radial, n=(1,nmax), l=(0,nmax-1))
output = interactive_plot.children[-1]
output.layout.height = '900px'
interactive_plot

interactive(children=(IntSlider(value=5, description='n', max=10, min=1), IntSlider(value=4, description='l', …

In [14]:
from bokeh.plotting import figure, show, output_file

N = 500
x = np.linspace(-20, 20, N)
y = np.linspace(-20, 00, N)
xx, yy = np.meshgrid(x, y)
r = np.sqrt(xx**2 + yy**2)
phi = np.arctan2(yy,xx)
density = prob_density(r,np.pi,phi,1,0,0)
print(density)

p = figure(x_range=(-20,20), y_range=(-20, 20),
           tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")])

# must give a vector of image data for image parameter
p.image(image=[density], x=0, y=0, dw=10, dh=10, palette="Spectral11")

#output_file("image.html", title="image.py example")

show(p)  # open a browser






[[2.01786252e-08 2.13542094e-08 2.25956996e-08 ... 2.25956996e-08
  2.13542094e-08 2.01786252e-08]
 [2.07583926e-08 2.19690030e-08 2.32475609e-08 ... 2.32475609e-08
  2.19690030e-08 2.07583926e-08]
 [2.13542094e-08 2.26008541e-08 2.39175492e-08 ... 2.39175492e-08
  2.26008541e-08 2.13542094e-08]
 ...
 [7.99162502e-05 8.65860648e-05 9.38125420e-05 ... 9.38125420e-05
  8.65860648e-05 7.99162502e-05]
 [7.99258792e-05 8.65965394e-05 9.38239366e-05 ... 9.38239366e-05
  8.65965394e-05 7.99258792e-05]
 [7.99290891e-05 8.66000312e-05 9.38277352e-05 ... 9.38277352e-05
  8.66000312e-05 7.99290891e-05]]


In [17]:
from ipywidgets import interactive, IntSlider, interact, BoundedIntText
n_widget = IntSlider(min=1, max=10, step=1, value=1)
l_widget = IntSlider(min=0, max=9, step=1)

def update_l_range(*args):
    l_widget.max = n_widget.value -1
n_widget.observe(update_l_range, 'value')

def printer(x, y):
    print(x, y)
interact(printer,x=n_widget, y=l_widget);

interactive(children=(IntSlider(value=1, description='x', max=10, min=1), IntSlider(value=0, description='y', …

In [27]:
n_value = BoundedIntText(
    value=1,
    min=0,
    max=10,
    step=1,
    description='n:',
    disabled=False
)
n_value

BoundedIntText(value=1, description='n:', max=10)

In [28]:
n_value.value

2